In [1]:
import torch
print('version', torch.__version__)
print(torch.cuda.is_available())

version 2.9.0+cu126
False


/home/atdehkordi/miniconda3/envs/pytorch/lib/python3.11/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
### Reading ForestCanopyHeight from AlphaEarth

import numpy as np
import pandas as pd

df = pd.read_excel('/home/atdehkordi/PHDLund/PythonProjects_github/VVR080F_ANN-DL/Data_Regression/ForestCanopyHeight/AlphaEarth/30m.xlsx')

x_inputs = df.iloc[:, 0:64].to_numpy(dtype = np.float32) 
y_output = df.iloc[:, -1].to_numpy(dtype = np.float32)

print('x shape:', x_inputs.shape)
print('y shape:', y_output.shape)

print('x min:', np.min(x_inputs))
print('x max:', np.max(x_inputs))

print('y min:', np.min(y_output))
print('y max:', np.max(y_output))

import matplotlib.pyplot as plt

plt.hist(y_output, bins=10)
plt.xlabel('FCH [m]')

In [ ]:
### If some ranges of the y target are having very low number of samples, here can be removed:

index = y_output<=40
y_output = y_output[index]
x_inputs = x_inputs[index, :]

print('y shape:', y_output.shape)
print('y min:', np.min(y_output))
print('y max:', np.max(y_output))

plt.hist(y_output, bins=10)
plt.xlabel('FCH [m]')

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

def log_scale_transform(X, Y):
    
    shift_value_X = np.abs(np.min(X)) + 1  # Shift X to ensure positive values
    shift_value_Y = np.abs(np.min(Y)) + 1  # Shift Y similarly
    
    X_shifted = X + shift_value_X
    Y_shifted = Y + shift_value_Y
    
    X_log = np.log(X_shifted)
    Y_log = np.log(Y_shifted)

    s1 = np.arange(X_log.shape[0])
    np.random.shuffle(s1)
    X_log = X_log[s1, :]
    Y_log = Y_log[s1]
    
    transformerX = RobustScaler().fit(X_log)
    X_trans = transformerX.transform(X_log)
    min_max_scalerX = MinMaxScaler().fit(X_trans)
    X_trans2 = min_max_scalerX.transform(X_trans)

    transformerY = RobustScaler().fit(np.reshape(Y_log,(-1,1)))
    Y_trans = transformerY.transform(np.reshape(Y_log,(-1,1)))
    min_max_scalerY = MinMaxScaler().fit(Y_trans)
    Y_trans2 = min_max_scalerY.transform(Y_trans)

    return X_trans2, Y_trans2, transformerX, transformerY, min_max_scalerX, min_max_scalerY, shift_value_X, shift_value_Y, s1

def ytest_to_initial_scale(ytest, min_max_scalerY_turb, transformerY_turb, shift_value_Y):
    ytest_inv1 = min_max_scalerY_turb.inverse_transform(np.reshape(ytest, (-1, 1)))
    ytest_inv2 = transformerY_turb.inverse_transform(ytest_inv1)
    ytest_inv2 = np.exp(ytest_inv2)
    ytest_rescaled = ytest_inv2 - shift_value_Y
    return ytest_rescaled

from scipy import stats

def r_squared(y, y_hat):
 ''' Logarithmic R^2 '''
 slope_, intercept_, r_value, p_value, std_err = stats.linregress(y, y_hat)
 return r_value**2 * 100

def mape(y, y_hat):
 ''' Mean Absolute Percentage Error '''
 return 100 * np.mean(np.abs((y - y_hat) / y))

In [ ]:
x_rescaled, y_rescaled, transformerX, transformerY, min_max_scalerX, min_max_scalerY, shift_value_X, shift_value_Y, s1 = log_scale_transform(x_inputs, y_output)
y_rescaled = np.squeeze(y_rescaled)

print('data_total,' , y_rescaled.shape)
print('min,' ,np.min(y_rescaled))
print('max,' ,np.max(y_rescaled))
print('mean,' ,np.mean(y_rescaled))
print('std,' ,np.std(y_rescaled))

plt.hist(y_rescaled, bins=10)
plt.xlabel('Transformed FCH [m]')

In [ ]:
### To see the performance of a simple RF

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV, LeaveOneOut, cross_val_predict
from sklearn.model_selection import train_test_split

# --------------------------
# 0) Putting n% of the data aside only for testing!
# --------------------------

test_size = 0.3

print("###################### Step 1")
print(f"Train and Test split: {test_size * 100}% of dataset is used only for testing...")

## If it is not your first run, activate the below parts for loading: 

file_path = '/home/atdehkordi/PHDLund/PythonProjects_github/VVR080F_ANN-DL/Data_Regression/ForestCanopyHeight/AlphaEarth/30m_train_test/'

X_train = np.load(file_path + 'X_train.npy')
X_test = np.load(file_path + 'X_test.npy')
y_train = np.squeeze(np.load(file_path + 'y_train.npy').reshape(-1,1))
y_test = np.squeeze(np.load(file_path + 'y_test.npy').reshape(-1,1))


## If it is your first run, comment the below parts for saving: 

# indices = np.arange(len(x_rescaled))

# X_train, X_test, y_train, y_test, train_idx, test_idx  = train_test_split(
#     x_rescaled, 
#     y_rescaled, 
#     indices,
#     test_size=test_size, 
#     random_state=42
# )

# file_path = '/home/atdehkordi/PHDLund/PythonProjects_github/VVR080F_ANN-DL/Data_Regression/ForestCanopyHeight/AlphaEarth/30m_train_test/'

# np.save(file_path + 'X_train.npy', X_train)
# np.save(file_path + 'X_test.npy', X_test)
# np.save(file_path + 'y_train.npy', y_train)
# np.save(file_path + 'y_test.npy', y_test)

# --------------------------
# 1) Tune hyperparameters using, say, 5-fold CV on training data
# --------------------------

print("###################### Step 2")
print(f"Hyperparameter tuning using k_fold validation on training data (100% - test_size = {100-test_size*100}%)...")

n_splits = 5

param_grid = {
    'n_estimators': [10, 50, 100, 200, 300],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 3, 4, 5, 6],
    'min_samples_leaf': [1, 2, 3, 4, 5]
}

rf_base = RandomForestRegressor(random_state=42)

kfold = KFold(n_splits= n_splits, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=kfold,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
print("Best hyperparameters using k-fold validation on training data:", grid_search.best_params_)

print("###################### Step 3")
print("Model validation using the best parameters found and k-fold method on training data...")

best_rf = RandomForestRegressor(n_estimators = grid_search.best_params_['n_estimators'],  
                                max_depth = grid_search.best_params_['max_depth'], 
                                min_samples_leaf = grid_search.best_params_['min_samples_leaf'], 
                                min_samples_split = grid_search.best_params_['min_samples_split'], 
                                random_state=42)


kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

mape_list_training = []
r2_list_training   = []

mape_list_test = []
r2_list_test  = []

counter = 1

for train_idx, test_idx in kf.split(X_train):

    print(f'######## Fold{counter}')
    counter += 1
    
    x_train_kfold, x_test_kfold = X_train[train_idx], X_train[test_idx]
    y_train_kfold, y_test_kfold = y_train[train_idx], y_train[test_idx]
    
    # Fit the best RF on the training fold
    best_rf.fit(x_train_kfold, y_train_kfold)
    
    # Predict on the test fold
    y_pred_rescaled_kfold = best_rf.predict(x_test_kfold)
    
    # Transform y_test and predictions back to original scale
    y_pred_original_kfold = np.squeeze(ytest_to_initial_scale(
        y_pred_rescaled_kfold, 
        min_max_scalerY, 
        transformerY, 
        shift_value_Y
    ))
    y_test_original_kfold = np.squeeze(ytest_to_initial_scale(
        y_test_kfold, 
        min_max_scalerY, 
        transformerY, 
        shift_value_Y
    ))

    # Compute metrics on THIS fold
    mape_list_training.append(mape(y_test_original_kfold, y_pred_original_kfold))
    r2_list_training.append(r_squared(y_test_original_kfold, y_pred_original_kfold))

    # Predict on the test fold
    y_pred_rescaled = best_rf.predict(X_test)
    
    # Transform y_test and predictions back to original scale
    y_pred_original = np.squeeze(ytest_to_initial_scale(
        y_pred_rescaled, 
        min_max_scalerY, 
        transformerY, 
        shift_value_Y
    ))
    y_test_original = np.squeeze(ytest_to_initial_scale(
        y_test, 
        min_max_scalerY, 
        transformerY, 
        shift_value_Y
    ))
    # Compute metrics on THIS fold
    mape_list_test.append(mape(y_test_original, y_pred_original))
    r2_list_test.append(r_squared(y_test_original, y_pred_original))

# Report average metrics across folds
print("Training : Mean MAPE (K-fold):", np.mean(mape_list_training))
print("Training : Std  MAPE (K-fold):", np.std(mape_list_training))
print("Training : Mean R²   (K-fold):", np.mean(r2_list_training))
print("Training : Std  R²   (K-fold):", np.std(r2_list_training))

# Report average metrics across folds
print("Test : Mean MAPE (K-fold):", np.mean(mape_list_test))
print("Test : Std  MAPE (K-fold):", np.std(mape_list_test))
print("Test : Mean R²   (K-fold):", np.mean(r2_list_test))
print("Test : Std  R²   (K-fold):", np.std(r2_list_test))

print("###################### Step 3")
print("Model validation using the best parameters found and whole training data on test data only")

best_rf = RandomForestRegressor(n_estimators = grid_search.best_params_['n_estimators'],  
                                max_depth = grid_search.best_params_['max_depth'], 
                                min_samples_leaf = grid_search.best_params_['min_samples_leaf'], 
                                min_samples_split = grid_search.best_params_['min_samples_split'], 
                                random_state=42)

best_rf.fit(X_train, y_train)

y_pred_rescaled = best_rf.predict(X_test)

y_pred_original = np.squeeze(ytest_to_initial_scale(
    y_pred_rescaled, min_max_scalerY, transformerY, shift_value_Y
))
y_test_original = np.squeeze(ytest_to_initial_scale(
    y_test, min_max_scalerY, transformerY, shift_value_Y
))

MAPE_value = mape(y_test_original, y_pred_original)
r2_value = r_squared(y_test_original, y_pred_original)

print("MAPE:", MAPE_value)
print("R2  :", r2_value)

In [ ]:
# For Chl-a plots

import matplotlib
from scipy.stats import gaussian_kde

filename = "Best_RF.jpg"

matplotlib.rcParams['axes.linewidth'] = 1.5

y = np.squeeze(y_pred_original)
x = np.squeeze(y_test_original)

mape_test = mape(np.squeeze(x), np.squeeze(y))
r2_test = r_squared(np.squeeze(x), np.squeeze(y))

print('')
print('MAPE = ', mape_test)
print('R2 = ', r2_test)
print('')

# Calculate the point density
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)

# Sort the points by density, so that the densest points are plotted last
idx = z.argsort()
x, y, z = x[idx], y[idx], z[idx]

fig, ax = plt.subplots(figsize = (10,10))

ax.plot([0,50], [0,50], color='grey', linestyle='dashed', linewidth=3) #for DO

ax.scatter(x, y, c=z, s=300, cmap='jet')

ax.set_xlim([0, 40])
ax.set_ylim([0, 40])

ax.set_xlabel("In-situ values", fontsize=20, fontweight="bold")
ax.set_ylabel("RF estimates", fontsize=20, fontweight="bold")

plt.xticks(fontsize=25, fontweight='bold');
plt.yticks(fontsize=25, fontweight='bold');

# fig.savefig(filename, dpi=700)

### In terms of training a self-attentin module, there is a need to define two terms:
1- tokens
2- embeddings

1-tokens: token is a unit that your model reads to understand a sequence. for example in NLP, words are sometimes tokens. In images, pathches of images can be considered as tokens.

2- an embedding is a vector that represents a token. Tokens are like IDs so if a token has an id of 25 and the other one is 26 there is no meaningful relationship between 25 and 26 like 26>25 but the embeddings carry meaningful informations about the tokens.

Tokenization is really important because if you consider a number for each word (token) then for the same word with different meaning, how you want to descriminate it. In other owrds, for the word leave in leave a message or leave the place, you have a similar token but they have different meanings. That is why you have to have both token and embedding. Token for cutting into smaller parts, embedding for meaning.
Embedding can also be determined as the word/token representation
Then for assessing the similarity, cosine of these embeddings is fine.

Self-attention understands relations between tokens using their embeddings.


When dealing with 1d data like a timeseries of a specific parameter or a spectral band of S2 data for WQ estimation, there is 2 ways to consider tokens and embeddings:

a) you can consider each feature as a token so for example you will have 10 tokens for 10 spectral bands of S2 data, each of it has only 1 embedding (band value)

    In this case you can also use a positional embedding so the attention also understands about the location of each feature.
    It is also recommended to increase the embedding size (not put it to 1) because it gives attention richer features to mix (add a small linear projection Linear(1 → d_model) to turn each scalar into a richer vector (e.g., d_model=32).)


b) you can consider you have 1 token and 64 features. in this case, transformers are useless. Because they are designed to extract the relations between tokens not embeddings.


### So, as a general rule: 
#### Self-attention only helps when there’s more than one token so  when your data has a sequence or structure across positions.

c) When I have for example 64 features, it is also possible to change it to a 8x8 2D token, where each feature becomes a token.


d) when you have a feature vector of 64 you can also make it 4x4 wit each embedding of 4 or, 2x2 with each embedding of 16 (in this case you can select different combination-infinite cases- so maybe it is a good idea to incorporate kmeans clustering but how: a very good idea for research)


### Now let's see how self attention works:

suppose that we have n_token=seq_len and n_emb=d_model
so: x.shape = (batch_size, seq_len, d_model)

Self-attention wants to say how strongly one token must look at the other tokens.
To do this, each token must send an information to the other tokens.
This is done by computing three projections:

Q=XWQ​,K=XWK​,V=XWV​ (Q = queries, K = keys, V = values)
In the above equation, Wq, Wk, Wv are learnable weights with size: d_model, d_k (we usually set: d_k = d_model/num_of_heads) where num_of_heads indicates how many self-attention modules we want to use.

as an example:

| Symbol              | Shape    | Meaning                                  |
| ------------------- | -------- | ---------------------------------------- |
| `X`                 | (64, 32) | 64 tokens, 32-dim embedding each         |
| `W_Q`, `W_K`, `W_V` | (32, 32) | projection weights                       |
| `Q`, `K`, `V`       | (64, 32) | 64 query/key/value vectors (1 per token) |

then, we need to compute similarities betwee Q and K:

Scores=QK.T (this is done using dot product which easily gives the similarities with somehow a cosine formula)
in the above example, Scores will be 64*64 (it gives the similarity of tokens all to each other or how much token i “attends to” token j before normalization.)


Then we apply softmax row-wise: A=softmax(​QKT​) Now each row A[i, :] sums to 1. 
A diagonal entry (A[i, i]) = how much a token looks at itself.
Off-diagonal (A[i, j]) = how much token i looks at token j.
Shape of A = (64, 64).

Now we weigh the value vectors (V) using attention weights A:
Output=AV Result: (64, 32) this is equivalent to initial X which was 64, 32 which means now for each token we have a new weighted embeddings


if you do a multi-head attention, like 8 heads: d_k = 32 / 8 = 4
so each head runs its own attention giving (64, 4) and then concatenates 8 heads to give (64, 32) output


So, When you have only one token (seq_len = 1) So nothing happens — the token only attends to itself with weight 1.
That’s why you need multiple tokens for attention to learn relationships.

If your input has 64 features and you treat each feature as a token (seq_len=64), So attention is literally learning which features are most related to which for predicting the target.


In the transformrs, there is also a term : positional embedding
because of the attention formula: Attention(Q,K,V)=softmax(QKT​)V, it does not inherently know which token came first, second, or last. So if you shuffled your tokens, you’d get the same output distribution — which is a problem for:
text (word order matters),
time series (temporal order matters),
spatial data (feature positions matter).
Therefore, Transformers need a way to inject positional information into the model.

So, we add (or sometimes concatenate) a position-dependent vector 

𝑋=𝑋+𝑃 where P is (seq, emb) meaning that the final output (x+P) encodes both:
its content (word/feature meaning)
its position (where it sits in the sequence)

Positional encoding methods:
1- Classical sinusoidal positional encoding
2- Learned positional embeddings

In [ ]:
#### simple self-attention for 1d data (each feature is considered as a single token so the embedding size will be 1,
#### I have also added a case with/without positional embedding (learnable/no-learnable) and also increasing the embedding size (yes/no)
#### and also to map the alphaearth fetures to 2D like 4*4 with 4 embeddings and so on.)

import torch
import torch.nn as nn

class SelfAttention(nn.Module):

    def __init__(self, seq_len = 64, attn_dropout = 0, hidden = 128, output_dim = 1):
        super().__init__()

        self.seq_len = seq_len # in case of using alphaearthnet you will have 64 features every of them with with one feature (embed=1)
        self.attn_dropout = attn_dropout
        self.hidden = hidden
        self.output_dim = output_dim
        
        self.attn = nn.MultiheadAttention(embed_dim=1, num_heads=1, dropout=attn_dropout, batch_first=True) # embed_dim means the dimensionality of both the input and output embeddings
        # MultiheadAttention genertaes the Q, K,V from inputs (it requires 3 inputs, if you give all 3, the Q, K, and V are computed from that)

        # Simple regression head
        self.mlp = nn.Sequential(
            nn.Flatten(),              # (B, 64, 1) -> (B, 64)
            nn.LayerNorm(seq_len),
            nn.Linear(seq_len, hidden),
            nn.GELU(),
            nn.Linear(hidden, output_dim)
        )

        # pass

    def forward(self, x):
        x = x.unsqueeze(-1)  # x is currenly (n_samples, n_features), I will make it (n_samples, n_features, 1) so n_features is the number of tokens and 1 is the embedding size
        
        attn_out, _ = self.attn(x, x, x)  # (B, 64, 1), self.attn gets three inputs of Q, V, and K where in our case they are all from data
        out = self.mlp(attn_out)          # (B, 1)
        return out
        
        # pass


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

import csv
import itertools
from sklearn.model_selection import ParameterGrid

hyperparameter_csv_file = 'SelfAttention_hyperparams.csv'

def write_csv(results_dict, csv_file):
    headers = list(results_dict.keys())
    rows = zip(*results_dict.values())

    with open(csv_file, mode='w', newline='') as f:
        writer = csv.writer(f)
        # Write the header row (dictionary keys)
        writer.writerow(headers)
        # Write each row of values
        for row in rows:
            writer.writerow(row)

param_grid = {
    'epochs': [100],
    'batch_size': [4, 8, 16, 32, 64, 128],
    'lr': [0.0005, 0.0001, 0.005, 0.001],
    'positional_encode':[0, 1, 2], # 0: None, 1: Sinsuidal, 2: Learnable,
    'increase_embedd_size': [0, 8], #0: None, any other number like 4, 8: change embed size to a higher dimension
}

param_grid = {
    'epochs': [100, 200, 300],
    'batch_size': [4, 8, 16, 32, 64, 128],
    'lr': [0.0005, 0.0001, 0.005, 0.001],
}

param_combinations = list(itertools.product(*param_grid.values()))

print("")
print(f"Number of all possible grid search scenarios: {len(list(ParameterGrid(param_grid)))}")

results = {key:[] for key in param_grid.keys()}
results['Train_MAPE'] = []
results['Train_R2'] = []
results['Test_MAPE'] = []
results['Test_R2'] = []

counter = 1

for params in param_combinations:

    print(f"\n####### Grid {counter}")
    counter = counter + 1

    params_dict = dict(zip(list(param_grid.keys()), params))

    train_loader = DataLoader(train_dataset, batch_size = params_dict['batch_size'], shuffle = True)
    test_loader = DataLoader(test_dataset, batch_size = params_dict['batch_size'], shuffle = True)

    model = SelfAttention()

    model.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr = params_dict['lr'])

    print("## Training")
    
    model.train()

    for epoch in range(params_dict['epochs']):
        for batch_x_train, batch_y_train in train_loader:
            batch_x_train, batch_y_train = batch_x_train.to(device), batch_y_train.to(device)
            optimizer.zero_grad()
            batch_y_predictions = model(batch_x_train)

            loss = criterion(batch_y_predictions, batch_y_train)

            loss.backward()
            optimizer.step()

    print("## Evaluation")

    model.eval()

    with torch.no_grad():

        predictions_test, truth_test, predictions_train, truth_train = [], [], [], []

        for batch_x_test, batch_y_test in test_loader:

            batch_x_test, batch_y_test = batch_x_test.to(device), batch_y_test.to(device)
            preds_test = model(batch_x_test)
            predictions_test.append(preds_test.cpu())
            truth_test.append(batch_y_test.cpu())

        predictions_test = torch.cat(predictions_test).numpy()
        predictions_test = ytest_to_initial_scale(predictions_test, min_max_scalerY, transformerY, shift_value_Y)
        truth_test = torch.cat(truth_test).numpy()
        truth_test = ytest_to_initial_scale(truth_test, min_max_scalerY, transformerY, shift_value_Y)

        mape_test = mape(np.squeeze(truth_test), np.squeeze(predictions_test))
        r2_test = r_squared(np.squeeze(truth_test), np.squeeze(predictions_test))

        for batch_x_train, batch_y_train in train_loader:

            batch_x_train, batch_y_train = batch_x_train.to(device), batch_y_train.to(device)
            preds_train = model(batch_x_train)
            predictions_train.append(preds_train.cpu())
            truth_train.append(batch_y_train.cpu())

        predictions_train = torch.cat(predictions_train).numpy()
        predictions_train = ytest_to_initial_scale(predictions_train, min_max_scalerY, transformerY, shift_value_Y)
        truth_train = torch.cat(truth_train).numpy()
        truth_train = ytest_to_initial_scale(truth_train, min_max_scalerY, transformerY, shift_value_Y)

        mape_train = mape(np.squeeze(truth_train), np.squeeze(predictions_train))
        r2_train = r_squared(np.squeeze(truth_train), np.squeeze(predictions_train))

        # Automate storing the parameter values and performance metrics
    for key, value in zip(list(param_grid.keys()), params):
        results[key].append(value)
    results["Test_MAPE"].append(mape_test)
    results["Test_R2"].append(r2_test)
    results["Train_MAPE"].append(mape_train)
    results["Train_R2"].append(r2_train)

    write_csv(results, hyperparameter_csv_file)
